In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#numpy.random.seed(0)
from keras.models import Model, load_model
from keras.layers import Input
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense, Dropout

#Sample data output
df=pd.read_csv("drive/My Drive/Datasets/Sign/sign_mnist_train.csv")
df.head()

Using TensorFlow backend.


FileNotFoundError: ignored

In [0]:
X = np.zeros((df.shape[0], 28, 28, 1), dtype=np.float32)
Y = np.zeros((df.shape[0],), dtype=np.int32)
for n in range(df.shape[0]):
  Y[n] = df['label'][n]
  X[n] = df.iloc[n,1:].values.reshape(28, 28, 1)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, stratify=Y, shuffle=True, random_state=42)

In [0]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoder.fit(Y.reshape(Y.shape[0], 1))

In [0]:
y_train = onehot_encoder.transform(y_train.reshape(y_train.shape[0], 1))
y_val = onehot_encoder.transform(y_val.reshape(y_val.shape[0], 1))

In [0]:
X_train = X_train/255
X_val = X_val/255

In [0]:
inputs = Input((28, 28, 1))
x = Conv2D(64, (3, 3), activation = 'relu', padding='same') (inputs)
x = MaxPooling2D(pool_size = (2, 2)) (x)
x = Conv2D(64, (3, 3), activation = 'relu', padding='same') (x)
x = MaxPooling2D(pool_size = (2, 2)) (x)
x = Conv2D(64, (3, 3), activation = 'relu', padding='same') (x)
x = MaxPooling2D(pool_size = (2, 2)) (x)
x = Flatten() (x)
x = Dense(units = 128, activation = 'relu') (x)
x = Dropout(0.20) (x)
y = Dense(units = 24, activation = 'softmax') (x)

model = Model(inputs=[inputs], outputs=[y])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [0]:
earlystopper = EarlyStopping(patience=10, verbose=1)
lr_changer = ReduceLROnPlateau(patience=3, verbose=1)
checkpointer = ModelCheckpoint('model.h5', verbose=1, save_best_only='True')

history = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=32, epochs=20, callbacks=[lr_changer, earlystopper, checkpointer])

In [0]:
preds = model.predict(X_val, batch_size=32, verbose=1)
preds = preds.argmax(axis=-1)
preds


In [0]:
confm = {'tp':0, 'fp':0}
for n in range(y_val.shape[0]):
  if y_val.argmax(axis=-1)[n]==preds[n]:
    confm['tp'] +=1
  if y_val.argmax(axis=-1)[n]!=preds[n]:
    confm['fp'] +=1
print(confm)

In [0]:

#Val accuracy
score, acc = model.evaluate(X_val, y_val,
                            batch_size=32)
print('Val score:', score)
print('Val accuracy:', acc)


In [0]:
#Sample data output
df2=pd.read_csv("drive/My Drive/Datasets/Sign/sign_mnist_test.csv")
df2.head()

In [0]:
X_test = np.zeros((df2.shape[0], 28, 28, 1), dtype=np.float32)
Y_test = np.zeros((df2.shape[0],), dtype=np.int32)
for n in range(df2.shape[0]):
  Y_test[n] = df2['label'][n]
  X_test[n] = df2.iloc[n,1:].values.reshape(28, 28, 1)

In [0]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoder.fit(Y_test.reshape(Y_test.shape[0], 1))

In [2]:
Y_test = onehot_encoder.transform(Y_test.reshape(Y_test.shape[0], 1))
X_test = X_test/255

NameError: ignored

In [0]:
Tpreds = model.predict(X_test, batch_size=32, verbose=1).argmax(axis=-1)
Tpreds

In [0]:
Tconfm = {'tp':0, 'fp':0}
for n in range(df2.shape[0]):
  if Y_test.argmax(axis=-1)[n]==Tpreds[n]:
    Tconfm['tp'] +=1
  if Y_test.argmax(axis=-1)[n]!=Tpreds[n]:
    Tconfm['fp'] +=1
print(Tconfm)

In [0]:
#Test accuracy
score, acc = model.evaluate(X_test, Y_test, batch_size=32)
print('Test score:', score)
print('Test accuracy:', acc)